In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

# Add a variable for the files to load
city_data_to_load = os.path.join("..","Weather_Database","WeatherPy_Database.csv")

# Read the City Data
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Torbay,CA,47.6666,-52.7314,81.37,78,40,3.44,light rain
1,1,Comodoro Rivadavia,AR,-45.8667,-67.5000,51.66,62,0,3.44,clear sky
2,2,Cape Town,ZA,-33.9258,18.4232,62.11,66,0,1.01,clear sky
3,3,Rikitea,PF,-23.1203,-134.9692,74.48,75,1,13.38,clear sky
4,4,Jamestown,US,42.0970,-79.2353,72.30,65,1,3.00,clear sky
5,5,Hobart,AU,-42.8794,147.3294,43.27,86,90,4.00,overcast clouds
6,6,Kieta,PG,-6.2167,155.6333,78.24,85,82,4.12,light rain
7,7,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,48.79,100,20,4.47,few clouds
8,8,East London,ZA,-33.0153,27.9116,61.74,82,0,10.36,clear sky
9,9,Esperance,AU,-33.8667,121.9000,49.84,87,0,8.08,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Torbay,CA,47.6666,-52.7314,81.37,78,40,3.44,light rain
3,3,Rikitea,PF,-23.1203,-134.9692,74.48,75,1,13.38,clear sky
4,4,Jamestown,US,42.0970,-79.2353,72.30,65,1,3.00,clear sky
6,6,Kieta,PG,-6.2167,155.6333,78.24,85,82,4.12,light rain
10,10,Great Bend,US,38.3645,-98.7648,78.40,94,75,9.22,broken clouds
12,12,Hirara,JP,24.8000,125.2833,82.89,80,0,11.21,clear sky
13,13,Moyale,KE,3.5167,39.0584,74.28,44,97,10.85,overcast clouds
18,18,Nakamura,JP,32.9833,132.9333,75.00,97,100,2.13,light rain
19,19,Castro,BR,-24.7911,-50.0119,70.09,59,82,5.86,broken clouds
21,21,Hilo,US,19.7297,-155.0900,72.63,88,1,5.75,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                265
City                   265
Country                264
Lat                    265
Lng                    265
Max Temp               265
Humidity               265
Cloudiness             265
Wind Speed             265
Current description    265
dtype: int64

In [6]:
preferred_cities_df=preferred_cities_df.dropna()

In [7]:
preferred_cities_df.count()

City_ID                264
City                   264
Country                264
Lat                    264
Lng                    264
Max Temp               264
Humidity               264
Cloudiness             264
Wind Speed             264
Current description    264
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Torbay,CA,81.37,light rain,47.6666,-52.7314,
3,Rikitea,PF,74.48,clear sky,-23.1203,-134.9692,
4,Jamestown,US,72.30,clear sky,42.0970,-79.2353,
6,Kieta,PG,78.24,light rain,-6.2167,155.6333,
10,Great Bend,US,78.40,broken clouds,38.3645,-98.7648,
12,Hirara,JP,82.89,clear sky,24.8000,125.2833,
13,Moyale,KE,74.28,overcast clouds,3.5167,39.0584,
18,Nakamura,JP,75.00,light rain,32.9833,132.9333,
19,Castro,BR,70.09,broken clouds,-24.7911,-50.0119,
21,Hilo,US,72.63,clear sky,19.7297,-155.0900,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
len(hotel_df)

264

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
print(len(clean_hotel_df))

242


In [12]:
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [13]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Torbay,CA,81.37,light rain,47.6666,-52.7314,Gallows Cove
3,Rikitea,PF,74.48,clear sky,-23.1203,-134.9692,People ThankYou
4,Jamestown,US,72.30,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
6,Kieta,PG,78.24,light rain,-6.2167,155.6333,Uruna Bay Retreat
10,Great Bend,US,78.40,broken clouds,38.3645,-98.7648,Best Western Angus Inn
12,Hirara,JP,82.89,clear sky,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
13,Moyale,KE,74.28,overcast clouds,3.5167,39.0584,La Tahzan Guest House
18,Nakamura,JP,75.00,light rain,32.9833,132.9333,Hotel Crown Hills Nakamura
19,Castro,BR,70.09,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
21,Hilo,US,72.63,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))